# Obtain info on Toronto neighborhoods

We will use Beautiful Soup package in Python to scrap the Wikipedia page and obtain the table containing information about neighborhoods in Toronto

In [10]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

First, we pull the Wikipedia webpage from the Internet and then use `BeautifulSoup` to parse the HTML code.

In [14]:
website_url = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')
soup = BeautifulSoup(website_url.text, 'lxml')

The first table on the webpage contain the desired information, which can be scrapped and store in the `my_table`:

In [15]:
my_table = soup.find('table')

We loop through each row of the table and pull out the contain in each cell.

In [43]:
Postcode = []
Borough = []
Neighborhood = []

row_idx = 0

for row in my_table.find_all('tr'):
    if row_idx > 0:
        cols = row.find_all('td')
        Postcode.append(cols[0].get_text().strip())
        Borough.append(cols[1].get_text().strip())
        Neighborhood.append(cols[2].get_text().strip())
    row_idx += 1
df = pd.DataFrame({'PostalCode':Postcode, 'Borough': Borough, 'Neighborhood' : Neighborhood})    

We remove all the rows whose Boroughs were not assigned.

In [44]:
df = df[df['Borough'] != 'Not assigned']

We combine all the rows that have the same postal code and borough together. The neighborhoods' name are concatenated and separated by a comma.

In [52]:
df = df.groupby(['PostalCode', 'Borough'])['Neighborhood'].apply(lambda x: ', '.join(x)).reset_index()

We check to see whether there is any borough whose neighborhood are not named (i.e., `Not assigned`). Fortunately, none happens 

In [54]:
df[df['Neighborhood'] == 'Not assigned']

,PostalCode,Borough,Neighborhood
